In [ ]:
import logging
import Metashape
import os

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    stream=logging.sys.stdout)
logger = logging.getLogger(__name__)  # Use __name__ to get the module's name

In [ ]:
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

In [ ]:
def find_files(folder, types):
    return [entry.path for entry in os.scandir(folder) if (entry.is_file() and os.path.splitext(entry.name)[1].lower() in types)]

# Parameter that need to be set.


In [ ]:
project = 'test_3'
base_folder = f'/Users/danielwilliams/Documents/cg_drone/{project}'

image_folder = f'{base_folder}/images'
output_folder = f'{base_folder}/exports'

# Start of the processing.
* Nothing needs to be set below this.

In [ ]:
photos = find_files(image_folder, [".jpg", ".jpeg", ".tif", ".tiff"])
print(len(photos))

# Creat the Document
* the document is the core object for the project.

In [ ]:
doc = Metashape.Document()
doc.save(output_folder + '/project.psx')

In [ ]:
chunk = doc.addChunk()

# Add Photos
* Add the photos to the chunk that is going to process.

In [ ]:
chunk.addPhotos(photos)
doc.save()

# Match Photos
### Perform image matching for the chunk frame.
### Parameters
* downscale (int) – Image alignment accuracy.
* generic_preselection (bool) – Enable generic preselection.
* reference_preselection (bool) – Enable reference preselection.
* reference_preselection_mode (ReferencePreselectionMode) – Reference preselection mode.
* filter_mask (bool) – Filter points by mask.
* mask_tiepoints (bool) – Apply mask filter to tie points.
* filter_stationary_points (bool) – Exclude tie points which are stationary across
images.
* keypoint_limit (int) – Key point limit.
* keypoint_limit_per_mpx (int) – Key point limit per megapixel.
* tiepoint_limit (int) – Tie point limit.
* keep_keypoints (bool) – Store keypoints in the project.
* pairs (list of (int, int) tuples) – User defined list of camera pairs to match.
* cameras (list of int) – List of cameras to match.
* guided_matching (bool) – Enable guided image matching.
* reset_matches (bool) – Reset current matches.
* subdivide_task (bool) – Enable fine-level task subdivision.
* workitem_size_cameras (int) – Number of cameras in a workitem.
* workitem_size_pairs (int) – Number of image pairs in a workitem.
* max_workgroup_size (int) – Maximum workgroup size.
* progress (Callable[[float], None]) – Progress callback.

In [ ]:
chunk.matchPhotos(keypoint_limit = 40000,
                  tiepoint_limit = 10000,
                  generic_preselection = True,
                  reference_preselection = True)
doc.save()

In [ ]:
# Align Camera
### Perform photo alignment for the chunk.
### Parameters
* cameras (list of int) – List of cameras to align.
* point_clouds (list of int) – List of point clouds to align.
* min_image (int) – Minimum number of point projections.
* adaptive_fitting (bool) – Enable adaptive fitting of distortion coefficients.
* reset_alignment (bool) – Reset current alignment.
* subdivide_task (bool) – Enable fine-level task subdivision.
* progress (Callable[[float], None]) – Progress callback.

In [ ]:
# alignPhotos was renamed to alignCameras... based on notes in the PDF...
chunk.alignCameras()
doc.save()

# Optimize Cameras
### Perform optimization of tie points / camera parameters.
### Parameters
* fit_f (bool) – Enable optimization of focal length coefficient.
* fit_cx (bool) – Enable optimization of X principal point coordinates.
* fit_cy (bool) – Enable optimization of Y principal point coordinates.
* fit_b1 (bool) – Enable optimization of aspect ratio.
* fit_b2 (bool) – Enable optimization of skew coefficient.
* fit_k1 (bool) – Enable optimization of k1 radial distortion coefficient.
* fit_k2 (bool) – Enable optimization of k2 radial distortion coefficient.
* fit_k3 (bool) – Enable optimization of k3 radial distortion coefficient.
* fit_k4 (bool) – Enable optimization of k3 radial distortion coefficient.
* fit_p1 (bool) – Enable optimization of p1 tangential distortion coefficient.
* fit_p2 (bool) – Enable optimization of p2 tangential distortion coefficient.
* fit_corrections (bool) – Enable optimization of additional corrections.
* adaptive_fitting (bool) – Enable adaptive fitting of distortion coefficients.
* tiepoint_covariance (bool) – Estimate tie point covariance matrices.
* progress (Callable[[float], None]) – Progress callback.

In [ ]:
chunk.optimizeCameras(fit_b1=True,
                      fit_b2=True,
                      fit_k4=True)
doc.save()

# Build Depth Maps
### Generate depth maps for the chunk.
### Parameters
* downscale (int) – Depth map quality.
* filter_mode (FilterMode) – Depth map filtering mode.
* cameras (list of int) – List of cameras to process.
* reuse_depth (bool) – Enable reuse depth maps option.
* max_neighbors (int) – Maximum number of neighbor images to use for depth map generation.
* subdivide_task (bool) – Enable fine-level task subdivision.
* workitem_size_cameras (int) – Number of cameras in a workitem.
* max_workgroup_size (int) – Maximum workgroup size.
* progress (Callable[[float], None]) – Progress callback

In [ ]:
chunk.buildDepthMaps(downscale = 2,
                     filter_mode = Metashape.MildFiltering)
doc.save()

# Locate Reflectance Panels
### Locate reflectance panels based on QR-codes.
### Parameters
* progress (Callable[[float], None]) – Progress callback.

In [ ]:
chunk.locateReflectancePanels()
doc.save()

# Calibrate Reflectance Panels
### Calibrate reflectance factors based on calibration panels and/or sun sensor.
### Parameters
* use_reflectance_panels (bool) – Use calibrated reflectance panels.
* use_sun_sensor (bool) – Apply irradiance sensor measurements.
* progress (Callable[[float], None]) – Progress callback.

In [ ]:
chunk.calibrateReflectance()
doc.save()

In [ ]:
# I'm not sure why we are checking this, maybe to ensure a different set worked?
has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation

In [ ]:
if has_transform:
    chunk.buildPointCloud()
    doc.save()

    chunk.buildDem(source_data=Metashape.PointCloudData)
    doc.save()

    chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
    doc.save()

In [ ]:
# export results
chunk.exportReport(output_folder + '/report.pdf')

if chunk.model:
    chunk.exportModel(output_folder + '/model.obj')

if chunk.point_cloud:
    chunk.exportPointCloud(output_folder + '/point_cloud.las', source_data = Metashape.PointCloudData)

if chunk.elevation:
    chunk.exportRaster(output_folder + '/dem.tif', source_data = Metashape.ElevationData)

if chunk.orthomosaic:
    chunk.exportRaster(output_folder + '/orthomosaic.tif', source_data = Metashape.OrthomosaicData)

print('Processing finished, results saved to ' + output_folder + '.')